<a href="https://colab.research.google.com/github/Anton80213/Machine_learning/blob/main/Exercise4-Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [1]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()


--2021-06-05 09:44:01--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.33.208, 142.250.65.80, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.33.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-06-05 09:44:02 (103 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-05.png', 'happy2-12.png', 'happy1-13.png', 'happy1-01.png', 'happy1-10.png', 'happy2-08.png', 'happy2-14.png', 'happy1-06.png', 'happy2-10.png', 'happy1-17.png']
['sad2-06.png', 'sad2-10.png', 'sad1-06.png', 'sad2-04.png', 'sad1-03.png', 'sad2-13.png', 'sad1-02.png', 'sad2-17.png', 'sad1-01.png', 'sad2-09.png']


In [24]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999
    from tensorflow import keras
    from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
    import tensorflow as tf
    
    class myCallback(tf.keras.callbacks.Callback):
         # Your Code
        def on_epoch_end(self, epoch, logs={}):
          if(logs.get('acc')>=DESIRED_ACCURACY):
             print("\nДостигнута точность 99.9%, обучение прерывается!")
             self.model.stop_training = True
    
    callbacks = myCallback()
    
    # Этот блок кода должен задать и скомпилировать модель. Пожалуйста, имейте в виду, что размер изображений 150 X 150
    model = tf.keras.models.Sequential([
        # Здесь ваш код
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])
        

     # Этот блок кода должен создать экземпляр ImageDataGenerator с именем train_datagen
      # и train_generator, вызовом train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)
    # Используйте target_size  150 X 150.
    train_generator = train_datagen.flow_from_directory(
        # Здесь ваш код #)
        '/tmp/h-or-s/',  # Это исходный каталог для тренировочных изображений
        target_size=(150, 150),  # все изображения будут изменены до 150x150
        batch_size=128,
        # поскольку мы используем функцию потерь binary_crossentropy, нам нужны двоичные метки
        class_mode='binary')
    # ожидаемая строка вывода: 'Found 80 images belonging to 2 classes'
    
    history = model.fit_generator(
          # Ваш код здесь )
    # model fitting
    train_generator,
    steps_per_epoch=1,  
    epochs=30,
    callbacks=[callbacks],
    verbose=1 
    )
    return history.history['acc'][-1]

In [25]:
# ожидаемая строка вывода: "Достигнута точность 99.9%, обучение прерывается!"
train_happy_sad_model()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 80 images belonging to 2 classes.
Epoch 1/30
1/1 [==============================] - 2s 2s/step - loss: 0.6912 - acc: 0.5000
Epoch 2/30
1/1 [==============================] - 2s 2s/step - loss: 5.9255 - acc: 0.5000
Epoch 3/30
1/1 [==============================] - 2s 2s/step - loss: 1.5013 - acc: 0.5000
Epoch 4/30
1/1 [==============================] - 2s 2s/step - loss: 0.6949 - acc: 0.5000
Epoch 5/30
1/1 [==============================] - 2s 2s/step - loss: 0.6595 - acc: 0.5000
Epoch 6/30
1/1 [==============================] - 2s 2s/step - loss: 0.6292 - acc: 0.5875
Epoch 7/30
1/1 [==============================] - 2s 2s/step - loss: 0.5829 - acc: 0.6500
Epoch 8/30
1/1 [==============================] - 2s 2s/step - loss: 0.6393 - acc: 0.5000
Epoch 9/30
1/1 [==============================] - 2s 2s/step - loss: 0.6450 - acc: 0.5000
Epoch 10/30
1/1 [==============================] - 2s 2s/step - loss: 0.4890 - acc: 0.8875
Epoch 11/30
1/1 [==============================] - 2s 2s/st

1.0